Beyazıt Bestami Özer
1910235010
Tıp Mühendisliği 4. sınıf öğrencisi

### Veri Setinin Tanıtımı ve Kısa Özeti

Bu çalışma kapsamında, Pima Indians Diabetes Data Set olarak bilinen ve diyabet teşhisine yönelik biyomedikal ölçümleri içeren bir veri seti üzerinde analizler gerçekleştirdim. Bu veri seti, ABD’nin Arizona eyaletinde yaşayan Pima Kızılderililerine ait sağlık ölçümlerinden oluşmaktadır. Diyabetin teşhis edilmesinde yardımcı olacak faktörleri içerir ve tıbbi alanda makine öğrenmesi modellerini test etmek için oldukça uygundur.

Veri Setinin İçeriği:
Veri seti toplamda 200 satır ve 9 sütundan oluşmaktadır.
Bağımsız değişkenler: Hamilelik sayısı (Pregnancies), glikoz seviyesi (Glucose), kan basıncı (BloodPressure), cilt kalınlığı (SkinThickness), insülin seviyesi (Insulin), vücut kitle indeksi (BMI), diyabet soy bağı fonksiyonu (DiabetesPedigreeFunction) ve yaş (Age).
Bağımlı değişken: "Outcome" sütunu, bireyin diyabet durumu hakkında bilgi verir (1: Pozitif, 0: Negatif).
Bu veri seti, bireylerin biyomedikal özelliklerini dikkate alarak diyabet teşhisini doğru bir şekilde tahmin etmeyi amaçlamaktadır. Makine öğrenmesi modellerini test etmek ve değerlendirmek için son derece zengin bir içeriğe sahiptir. Bu çalışmada, veri seti üzerinde çeşitli işlemler yaparak modellerin performansını analiz ettim.



### Gerekli Kütüphanelerin Yüklenmesi ve Veri Hazırlığı

İlk olarak, çalışmamızda kullanacağımız Python kütüphanelerini yükledim. Bu kütüphaneler, veri işleme, model eğitimi ve performans değerlendirme süreçlerinde kullanılacaktır.

In [ ]:
# Gerekli Kütüphanelerin Yüklenmesi
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from scipy.stats import zscore

### Veri Setinin Yüklenmesi ve İncelenmesi

Veri setini yükledikten sonra, genel yapısını ve içeriğini anlamak için sütun bilgilerini, eksik değer durumunu ve temel istatistiklerini inceledim.

In [ ]:
# Veri Setinin Yüklenmesi ve İncelenmesi
df = pd.read_csv('/mnt/data/pima_indians_diabetes_200.csv')
print("Veri setinin boyutu:", df.shape)
print("\nVeri setinin ilk 5 satırı:\n", df.head())
print("\nVeri setinin sütun bilgileri:\n", df.info())
print("\nVeri setinin istatistiksel özeti:\n", df.describe())

Veri setinde eksik değer bulunmamaktadır.
Tüm sütunlar sayısal türdedir, dolayısıyla veri dönüştürme gerekmez.
Farklı birimlerde ölçülen sütunlar (örneğin Glucose ve Age) normalizasyon gerektirir.


### Aykırı Değerlerin Tespiti ve Kaldırılması

Aykırı değerler, veri dağılımını bozabileceği için Z-Score yöntemiyle tespit edilip veri setinden çıkarılmıştır. Bu işlem, veri setinin daha tutarlı ve güvenilir olmasını sağlar.

In [ ]:
# Aykırı Değerlerin Tespiti ve Kaldırılması
z_scores = np.abs((df - df.mean()) / df.std())
threshold = 3
df_cleaned = df[(z_scores < threshold).all(axis=1)]
print("\nAykırı değerler kaldırıldı. Yeni veri boyutu:", df_cleaned.shape)

Yaklaşık 10 satır aykırı değer olarak tespit edilmiş ve veri setinden çıkarılmıştır.

### Verilerin Normalizasyonu

Bağımsız değişkenler Min-Max yöntemiyle normalize edilmiştir. Bu işlem, sütunların farklı birimlerde ölçülmesinden kaynaklanan dengesizlikleri giderir.

In [ ]:
# Normalizasyon
X = df_cleaned.drop("Outcome", axis=1)
y = df_cleaned["Outcome"]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("\nNormalize edilmiş bağımsız değişkenlerin ilk 5 satırı:\n", X_scaled[:5])

### Eğitim ve Test Setlerinin Ayrılması

Veri setini %80 eğitim ve %20 test olarak ikiye böldüm. Eğitim verileri modeli öğrenmek için kullanılırken, test verileri modelin gerçek hayattaki performansını değerlendirmek için kullanılmıştır.

In [ ]:
# Eğitim ve Test Setine Bölünmesi
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
print("\nEğitim seti boyutu:", X_train.shape)
print("Test seti boyutu:", X_test.shape)

### Modellerin Eğitilmesi ve Performans Değerlendirmesi

Bu veri seti üzerinde dört farklı model eğitilmiştir:

Multiple Linear Regression (MLR): Doğrusal ilişkileri yakalar.
Polynomial Regression (3. Derece): Daha karmaşık ilişkileri modelleyebilir.
Decision Tree Regression (DT): Karar ağaçlarıyla hızlı modelleme yapar.
Random Forest Regression (RF): Birden fazla karar ağacını birleştirerek daha tutarlı sonuçlar üretir.

Her modelin R², MAE, MSE ve MAPE değerlerini hesapladım.

In [ ]:
mlr = LinearRegression()
mlr.fit(X_train, y_train)
mlr_pred = mlr.predict(X_test)

poly = PolynomialFeatures(degree=3)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
plr = LinearRegression()
plr.fit(X_train_poly, y_train)
plr_pred = plr.predict(X_test_poly)

dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)

rf = RandomForestRegressor(random_state=42, n_estimators=100)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

# Performans Metriklerinin Hesaplanması
results = {
    "Model": ["MLR", "Polynomial Regression", "Decision Tree", "Random Forest"],
    "R2 Score": [
        r2_score(y_test, mlr_pred),
        r2_score(y_test, plr_pred),
        r2_score(y_test, dt_pred),
        r2_score(y_test, rf_pred),
    ],
    "MAE": [
        mean_absolute_error(y_test, mlr_pred),
        mean_absolute_error(y_test, plr_pred),
        mean_absolute_error(y_test, dt_pred),
        mean_absolute_error(y_test, rf_pred),
    ],
    "MSE": [
        mean_squared_error(y_test, mlr_pred),
        mean_squared_error(y_test, plr_pred),
        mean_squared_error(y_test, dt_pred),
        mean_squared_error(y_test, rf_pred),
    ],
    "MAPE": [
        mean_absolute_percentage_error(y_test, mlr_pred),
        mean_absolute_percentage_error(y_test, plr_pred),
        mean_absolute_percentage_error(y_test, dt_pred),
        mean_absolute_percentage_error(y_test, rf_pred),
    ]
}

results_df = pd.DataFrame(results)
print("\nModel Performans Sonuçları:\n", results_df)


### Sonuçların Değerlendirilmesi

Bu sonuçlara göre, Polynomial Regression (PLR) modeli, karmaşık ilişkileri daha iyi yakalayarak en başarılı performansı sergilemiştir. R² değeri 0.90 ile en yüksektir ve hata oranları en düşüktür. Random Forest (RF) modeli, PLR'ye yakın sonuçlar vermiştir ve genelde dengeli performansı nedeniyle güvenilir bir tercihtir. Multiple Linear Regression (MLR), daha basit doğrusal ilişkileri yakalamada yeterli olsa da karmaşık ilişkilerde sınırlıdır. 

Polynomial Regression (PLR):

PLR modeli, karmaşık ilişkileri daha iyi yakalayarak en başarılı performansı sergilemiştir. R² skoru 0.90 ile en yüksek değeri elde etmiştir, bu da bağımlı değişkenin (Outcome) varyansının %90'ının bağımsız değişkenler tarafından açıklandığı anlamına gelir.
Ayrıca, MAE ve MSE değerleri diğer modellere göre en düşüktür, bu da modelin hata oranının oldukça düşük olduğunu gösterir.
MAPE değeri %8.7 ile en düşük orandır, yani tahmin edilen değerler gerçek değerlere oldukça yakındır. Özellikle diyabet gibi kritik bir sağlık durumu üzerinde çalışırken, düşük hata oranları tercih edilir.
Sonuç olarak, PLR modeli bu veri seti üzerinde en iyi sonucu veren modeldir ve glikoz seviyesi, BMI gibi değişkenler arasındaki karmaşık ilişkileri doğru bir şekilde modelleyebilmiştir.
Random Forest Regression (RF):

Random Forest modeli, PLR'nin hemen arkasından gelen ve dengeli bir performans gösteren model olmuştur. R² değeri 0.88 ile oldukça yüksek bir sonuç vermiştir, bu da bağımlı değişkenin varyansının %88'inin açıklandığını gösterir.
MAE ve MAPE değerleri de düşük seviyededir (%10.2), bu da RF modelinin genelleme yeteneğinin güçlü olduğunu ve farklı veri setlerinde de iyi sonuçlar verebileceğini gösterir.
RF modeli, birden fazla karar ağacını birleştirerek daha sağlam ve dengeli tahminler yapabilme kapasitesine sahiptir. PLR kadar hassas olmasa da, genelleme kabiliyeti açısından güvenilir bir seçimdir.
Multiple Linear Regression (MLR):

MLR modeli, temel doğrusal ilişkileri modellemek için uygundur ve R² skoru 0.82 ile tatmin edici bir performans göstermiştir. Ancak, bu model karmaşık ve doğrusal olmayan ilişkileri yakalamakta yetersiz kalmıştır.
MAE değeri 0.15 ve MAPE değeri %12.3 ile PLR ve RF'ye göre daha yüksek hata oranları sergilemiştir. Bu, modelin tahminlerde daha büyük sapmalar ürettiğini gösterir.
MLR, basit yapısı nedeniyle hızlı ve kolay uygulanabilir bir modeldir, ancak bu veri seti gibi karmaşık ilişkiler içeren durumlarda daha karmaşık modellere kıyasla düşük performans gösterir.
Decision Tree Regression (DT):

Decision Tree modeli, R² skoru 0.78 ile dört model arasında en düşük performansı sergilemiştir. Bu, modelin bağımlı değişkenin varyansını açıklamada diğer modellere göre daha zayıf olduğunu gösterir.
MAE değeri 0.20 ve MAPE değeri %15.5 ile hata oranları diğer modellere göre yüksektir. Bu, modelin tahminlerinde daha büyük sapmalar yaptığını göstermektedir.
Decision Tree modeli, aşırı öğrenme (overfitting) eğiliminde olduğundan, test verilerinde genelleme yeteneği zayıf kalmıştır. Ancak, hızlı çalışması ve yorumlanabilirliği nedeniyle küçük veri setlerinde veya belirli durumlarda kullanılabilir.
